In [78]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
from scipeds.data.queries import QueryFilters, TaxonomyRollup
from scipeds.data.completions import CompletionsQueryEngine
from scipeds.data.enums import FieldTaxonomy

from pathlib import Path

import pandas as pd

In [80]:
db_path = Path('data/processed/ipeds.duckdb')

In [81]:
engine = CompletionsQueryEngine(db_path=db_path)

In [82]:
qf = QueryFilters(start_year=1984, end_year=1994)


First let's diagnose the problem -- how many of the CIP codes (converted to their cip2020 version) in the 1984-1994 data are unknown / not in NCSES?

In [83]:
df = engine.field_totals_by_grouping(grouping='gender', query_filters=qf, taxonomy=FieldTaxonomy.cip, by_year=True)
df = df.reset_index()
df.head()

,FieldTaxonomy.cip,gender,year,field_degrees_within_gender,field_degrees_total,uni_degrees_within_gender,uni_degrees_total
0,01.0000,men,1984,1779,2354,983227,1991889
1,01.0000,men,1985,1649,2222,980838,2004285
2,01.0000,men,1986,1451,1974,975329,2008295
3,01.0000,men,1987,1218,1705,1036892,2163263
4,01.0000,men,1988,1192,1644,1112371,2324652


In [84]:
cipdf = engine.get_cip_table()
cipdf[cipdf['cip_title'] == "Unknown"]

,cip_title,ncses_sci_group,ncses_field_group,ncses_detailed_field_group,nsf_broad_field,dhs_stem
cip2020,,,,,,
18.0403,Unknown,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Non-science and engineering,False
18.0408,Unknown,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Non-science and engineering,False
18.1501,Unknown,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Non-science and engineering,False
07.0607,Unknown,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Non-science and engineering,False
17.0512,Unknown,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Non-science and engineering,False
...,...,...,...,...,...,...
17.0805,Unknown,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Non-science and engineering,False
28.0201,Unknown,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Non-science and engineering,False
18.1029,Unknown,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Not categorized in NCSES crosswalk,Non-science and engineering,False


In [85]:
df_w_cip = pd.merge(
    df, cipdf,
    left_on=FieldTaxonomy.cip, left_index=False,
    right_index=True, 
    validate='many_to_one'
)
df_w_cip.head()

,FieldTaxonomy.cip,gender,year,field_degrees_within_gender,field_degrees_total,uni_degrees_within_gender,uni_degrees_total,cip_title,ncses_sci_group,ncses_field_group,ncses_detailed_field_group,nsf_broad_field,dhs_stem
0,01.0000,men,1984,1779,2354,983227,1991889,"Agriculture, Agriculture Operations And Relate...",Science and engineering,Life Sciences,Agricultural Sciences,Agricultural and biological sciences,False
1,01.0000,men,1985,1649,2222,980838,2004285,"Agriculture, Agriculture Operations And Relate...",Science and engineering,Life Sciences,Agricultural Sciences,Agricultural and biological sciences,False
2,01.0000,men,1986,1451,1974,975329,2008295,"Agriculture, Agriculture Operations And Relate...",Science and engineering,Life Sciences,Agricultural Sciences,Agricultural and biological sciences,False
3,01.0000,men,1987,1218,1705,1036892,2163263,"Agriculture, Agriculture Operations And Relate...",Science and engineering,Life Sciences,Agricultural Sciences,Agricultural and biological sciences,False
4,01.0000,men,1988,1192,1644,1112371,2324652,"Agriculture, Agriculture Operations And Relate...",Science and engineering,Life Sciences,Agricultural Sciences,Agricultural and biological sciences,False


In [86]:
# How many unique unclassified CIPs per year?
df_w_cip[df_w_cip['cip_title'] == "Unknown"][['year', FieldTaxonomy.cip]].drop_duplicates().groupby('year').size()

year
1984    335
1985    352
1986    343
1987    284
1988    276
1989    280
1990    403
1991    394
1992      1
1993      1
1994      1
dtype: int64

In [87]:
# How many total CIPs per year?
df_w_cip[['year', FieldTaxonomy.cip]].drop_duplicates().groupby('year').size()

year
1984    968
1985    988
1986    977
1987    938
1988    927
1989    931
1990    953
1991    941
1992    829
1993    829
1994    825
dtype: int64

Ok so for earlier years it's like half of the CIPs that aren't classified.

We need to figure out why these CIPs aren't being classified. Likely becuase they're not in the 85 -> 90 classifier?

In [88]:
unknown_cips = df_w_cip[df_w_cip['cip_title'] == "Unknown"][['year', FieldTaxonomy.cip]].drop_duplicates()
unknown_cips.head()

,year,FieldTaxonomy.cip
176,1985,01.0202
177,1984,01.0203
178,1985,01.0203
179,1986,01.0203
180,1987,01.0203


In [89]:
unknown_cips[unknown_cips[FieldTaxonomy.cip] == "01.0202"]

,year,FieldTaxonomy.cip
176,1985,01.0202


01.0202
- listed as "10202 - Agricultural Electrification Power and Controls" in the 1985 data dictionary.
- Only has one degree?

Let's see how many _students_ these unknowns represent per year

In [90]:
df_w_cip[df_w_cip['cip_title'] == "Unknown"][['year', FieldTaxonomy.cip, "field_degrees_total"]].drop_duplicates().groupby('year')["field_degrees_total"].sum()

year
1984     89754
1985     89565
1986     92967
1987     94513
1988    189860
1989    245007
1990    844741
1991    887294
1992     51136
1993     30515
1994     20746
Name: field_degrees_total, dtype: int64

Uhhh wait is that _all_ of the degrees in 1984?

In [91]:
df_w_cip[['year', 'uni_degrees_total']].drop_duplicates()

,year,uni_degrees_total
0,1984,1991889
1,1985,2004285
2,1986,2008295
3,1987,2163263
4,1988,2324652
5,1989,2412546
6,1990,2320375
7,1991,2408979
8,1992,2550460
9,1993,2616641


In [92]:
proportion_missing = pd.merge(
    df_w_cip[['year', 'uni_degrees_total']].drop_duplicates(),
    df_w_cip[df_w_cip['cip_title'] == "Unknown"][['year', FieldTaxonomy.cip, "field_degrees_total"]].drop_duplicates().groupby('year')["field_degrees_total"].sum(),
    right_index=True,
    left_on='year'
).rename(columns={'field_degrees_total': 'unknown_cip_total'})

proportion_missing['fraction_unknown_cips'] = proportion_missing['unknown_cip_total'] / proportion_missing['uni_degrees_total']

proportion_missing

,year,uni_degrees_total,unknown_cip_total,fraction_unknown_cips
0,1984,1991889,89754,0.045060
1,1985,2004285,89565,0.044687
2,1986,2008295,92967,0.046292
3,1987,2163263,94513,0.043690
4,1988,2324652,189860,0.081672
5,1989,2412546,245007,0.101555
6,1990,2320375,844741,0.364054
7,1991,2408979,887294,0.368328
8,1992,2550460,51136,0.020050
9,1993,2616641,30515,0.011662


Ok, let's start with the biggest unknowns for 84-86 and see if we can find them.

In [93]:
(
    df_w_cip[
        (df_w_cip['cip_title'] == "Unknown") & (df_w_cip['year'].isin([1984, 1985, 1986]))
    ]
    .groupby([FieldTaxonomy.cip])
    ['field_degrees_total']
    .sum()
).sort_values(ascending=False)

/var/folders/87/tp5jbk1x7zx0c0tx8zqvrxjr0000gp/T/ipykernel_70899/937130514.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby([FieldTaxonomy.cip])


FieldTaxonomy.cip
07.0305    46488
17.0602    43586
43.0105    28180
07.0608    27320
15.0302    20248
           ...  
16.0403        0
16.0402        0
16.0400        0
16.0399        0
95.9500        0
Name: field_degrees_total, Length: 1653, dtype: int64

06.0401 is in the CIP85toCIP90 crosswalk! 

`06.0401	Business Administration	52.0201	Business Administration & Management, General`

Oh, we just weren't reading in that crosswalk. Let's try adding that to the code and then...

Hm, now 06.0401 is being mapped correctly to its 1990 CIP code of 52.0201 but nothing after that.

In [94]:
cipdf.loc['52.0201']

cip_title                     Business Administration and Management, General
ncses_sci_group                                   Non-science and engineering
ncses_field_group                                     Business and Management
ncses_detailed_field_group                            Business and Management
nsf_broad_field                                   Non-science and engineering
dhs_stem                                                                False
Name: 52.0201, dtype: object

Ok it's being mapped from 1985 to 1990 correctly, but then it's not present in the 1990 crosswalk. Does that mean it should just be propagated? Or it needs to be in the dict?

Update: it was becuase the NCSES classifier was only classifying the original CIP code, not the 2020 one. And this original cip code (1) doesn't have a cip title (bc the original crosswalk file didn't have one) and (2) isn't in the NCSES classifier.

Ok so now we still have a bunch of missing codes, let's dig into those.

Update: the vast majority were "total" cips that weren't being seen. I fixed that.


Now it looks like the early 90's are most problematic. What's going on here? Let's look at 07.0305 as an example...

In [95]:
cipdf.loc["07.0305"]

cip_title                                                Unknown
ncses_sci_group               Not categorized in NCSES crosswalk
ncses_field_group             Not categorized in NCSES crosswalk
ncses_detailed_field_group    Not categorized in NCSES crosswalk
nsf_broad_field                      Non-science and engineering
dhs_stem                                                   False
Name: 07.0305, dtype: object

07.0305 Business Data Programming (from [here](https://nces.ed.gov/pubs91/91396.pdf))

Not finding it in the NSF data or in the Excel with all the crosswalks. Uh oh...